<a href="https://colab.research.google.com/github/hatch254/python/blob/main/ieee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

READ DATASET

In [4]:
import pandas as pd
import numpy as nm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.feature_selection import RFECV
import lightgbm as lgb
from tqdm import tqdm_notebook
from sklearn.metrics import roc_auc_score
import multiprocessing

import re
import gc
import warnings
warnings.filterwarnings('ignore')
warnings.warn("this will not show")

%matplotlib inline

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = 1000

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
!ls /content/drive/MyDrive

 4-Modes-Operations.gslides
'AUTHENTICATION APPLICATIONS.gdoc'
'Authentication Applications.gslides'
 Bachelors_Cert.pdf
'BBIT Year 4 2019-2020.gdoc'
'BBIT Year 4 2019-2020.pdf'
'BPM Group1'
 Classroom
 Cloud_Computing.docx
 Colab_Notebooks
'EVOLUTION OF THE INDUSTRIAL AGE.gslides'
'Net Zero Project.gslides'


In [7]:
!ls /content/drive/MyDrive/Colab_Notebooks/IEEE/train_transaction.csv

/content/drive/MyDrive/Colab_Notebooks/IEEE/train_transaction.csv


In [8]:
#FOR SPEED WE WILL COPY THE FILES TO THE VM OF GOOGLE COLAB IN THE DRIVE/DISK SPACE GIVEN
#WE WILL NAME THIS FOLDER AS /CONTENT/INPUT .. THE DESTINATION DIRECTORY
#WE ARE USING THE OS LIBRARY TO READ AND WRITE THE GOOGLE COLAB RESOURCES ENVIRONMENT

%%time

import os, shutil, zipfile

#configuring/initializing the source folder in Drive
source_dir = "/content/drive/MyDrive/Colab_Notebooks/IEEE"

#the four expected files in the source folder
files = ['test_identity.csv',
         'test_transaction.csv',
         'train_identity.csv',
         'train_transaction.csv']

#configuring the vm directory
destination_dir = "/content/input"
os.makedirs(destination_dir, exist_ok=True)

#------- verify each file existance in Drive --------------
def missing(source_dir, files):
  missing =[] #this is a null or empty list
  for name in files:
    source_path = os.path.join(source_dir, name)

    #check existence of file in path if truthy
    if not os.path.exists(source_path):
      missing.append(source_path)

  #checks if missing list is not empty
  if missing:
    raise FileNotFoundError(
        "The following file is not found int Drive:\n- "+"\n-".join(missing)
        +"\n\n Check the path in SRC_DIR and the filename in the files.")

  print("All source files are found in Drive")

missing(source_dir, files)

#------ Copy from Drive to the local vm for faster input/output ---------
for name in files:
  src = os.path.join(source_dir, name)
  dst = os.path.join(destination_dir, name)

  if not os.path.exists(dst): #check existence of file in path if truthy
    shutil.copy2(src, dst)
    print(f"Copied -> {dst}")
  else:
    print(f"Already exists -> {dst}")

# ----------- Now read fast from the vm local disk ------------
def read_csv_auto(path: str) -> pd.DataFrame:
  #If path ends with .zip and contains a single csv, read the csv.
  #If path ends with .csv, read directly.
  if path.lower().endswith(".zip"):
    with zipfile.ZipFile(path) as zf:
      #first find the csv inside the zip
      csv_files = [m for m in zf.namelist() if m.lower().endswith(".csv")]
      if not csv_files:
        raise ValueError(f"No CSV file in ZIP: {path}")

      #Prefer a file that matches the zip basename, fallback to the first csv
      base = os.path.splitext(os.path.basename(path))[0].lower()
      pick = None
      for m in csv_files:
        if base in os.path.basename(m).lower():
          pick = m
          break
      if pick is None:
        pick = csv_files[0]
      with zf.open(pick) as f:
        return pd.read_csv(f, low_memory=True)
  else:
    return pd.read_csv(path, low_memory=True)


#--------Load into DataFrames..........
local_paths = {name: os.path.join(destination_dir, name) for name in files}

print ("\n Loading DAtaFrames from /content ... ")
test_id = read_csv_auto(local_paths["test_identity.csv"])
test_tr = read_csv_auto(local_paths["test_transaction.csv"])
train_id = read_csv_auto(local_paths["train_identity.csv"])
train_tr = read_csv_auto(local_paths["train_transaction.csv"])

All source files are found in Drive
Copied -> /content/input/test_identity.csv
Copied -> /content/input/test_transaction.csv
Copied -> /content/input/train_identity.csv
Copied -> /content/input/train_transaction.csv

 Loading DAtaFrames from /content ... 
CPU times: user 52.9 s, sys: 20.7 s, total: 1min 13s
Wall time: 1min 45s


In [10]:
#The .shape returns a tuple in the format (number of rows, number of columns)
# A tuple is a number/dimensions of a dataframe

print("\n Shapes ")
print("test_id", test_id.shape)
print("test_tr", test_tr.shape)
print("train_id", train_id.shape)
print("train_tr", train_tr.shape)


 Shapes 
test_id (141907, 41)
test_tr (506691, 393)
train_id (144233, 41)
train_tr (590540, 394)


In [11]:
train = train_tr.merge(train_id, how='left', on='TransactionID')
test = test_tr.merge(test_id, how='left', on='TransactionID')

print("train", train.shape)
print("test", test.shape)

#Deleting the rows that are not needed
del(train_id, train_tr, test_id, test_tr)
#gabbage collector. Collects the deleted data so as to free up RAM memory
gc.collect()

train (590540, 434)
test (506691, 433)


154

In [15]:
print(f"The train dataset has {train.shape[0]} rows and {train.shape[1]} columns")
print(f"The test dataset has {test.shape[0]} rows and {test.shape[1]} columns")



The train dataset has 590540 rows and 434 columns
The test dataset has 506691 rows and 433 columns
